The purpose of this notebook is to use transfer learning in order to predict the Culture of the artwork from MET given images. In order to do that, we use GPU offered by Google Colab


# Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from google.colab import drive

The dataset is stored in google drive

In [ ]:
#get access to google colab
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
"""
The dataset is stored in a zip file 
"""
zip_file = '/content/gdrive/MyDrive/MUSEUM_IMAGES.zip' #path of the dataset

Run the following cell if and only if you have not yet unzip your dataset

In [ ]:
import zipfile
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('/content/gdrive/MyDrive')

In the following cell, we create the train, validation and test dataset

In [ ]:
batch_size = 64
IMG_SIZE = (224, 224) #specific for VGG16
IMG_SHAPE = IMG_SIZE + (3,)

file_data = 'MUSEUM_IMAGES'
data_dir_train = os.path.join('/content/gdrive/MyDrive', file_data, 'TRAIN')

train_data = tf.keras.utils.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="training",
  image_size= IMG_SIZE,
  seed=1,
  batch_size= batch_size)


val_data = tf.keras.utils.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="validation",
  seed=1,
  image_size= IMG_SIZE,
  batch_size= batch_size)



data_dir_test_unbalanced = os.path.join('/content/gdrive/MyDrive',file_data, 'TEST_UNBALANCED')
test_data_unbalanced = tf.keras.utils.image_dataset_from_directory(
  data_dir_test_unbalanced,
  image_size= IMG_SIZE,
  seed=1,
  batch_size= batch_size)


Found 7153 files belonging to 9 classes.
Using 5723 files for training.
Found 7153 files belonging to 9 classes.
Using 1430 files for validation.
Found 1789 files belonging to 9 classes.
Found 200 files belonging to 9 classes.


We import VGG16 in order to do transfer learning for our specific problem

In [ ]:
base_model = VGG16(input_shape=IMG_SHAPE , weights='imagenet', include_top=False)

# don't train existing weights
for layer in base_model.layers:
    layer.trainable = False
  

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input
rescale = tf.keras.layers.Rescaling(scale=1./255)
flatten = tf.keras.layers.Flatten()

We build our model. We change the last dense layer by putting a 4-dense layer with a softmax as activation function in order to achieve our classification problem

In [ ]:
num_classes = 9
x = Flatten()(base_model.output)
prediction = Dense(num_classes, activation='softmax')(x)

# create a model object
model = Model(inputs=base_model.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
from tensorflow.keras import optimizers
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer=optimizers.Adam(learning_rate=0.01),
  metrics=['accuracy']
)


model.fit( 
    train_data,
    validation_data=val_data,
    epochs=20,
)

Epoch 1/20
90/90 [==============================] - 90s 830ms/step - loss: 174.5379 - accuracy: 0.3935 - val_loss: 178.4415 - val_accuracy: 0.4245
Epoch 2/20
90/90 [==============================] - 71s 764ms/step - loss: 61.5926 - accuracy: 0.6991 - val_loss: 204.1411 - val_accuracy: 0.4273
Epoch 3/20
90/90 [==============================] - 71s 766ms/step - loss: 33.7190 - accuracy: 0.8164 - val_loss: 209.5868 - val_accuracy: 0.4566
Epoch 4/20
90/90 [==============================] - 70s 761ms/step - loss: 23.0884 - accuracy: 0.8614 - val_loss: 225.5429 - val_accuracy: 0.4699
Epoch 5/20
90/90 [==============================] - 70s 761ms/step - loss: 14.9745 - accuracy: 0.8983 - val_loss: 234.6190 - val_accuracy: 0.4608
Epoch 6/20
90/90 [==============================] - 70s 759ms/step - loss: 10.9408 - accuracy: 0.9236 - val_loss: 269.4076 - val_accuracy: 0.4699
Epoch 7/20
90/90 [==============================] - 70s 760ms/step - loss: 9.9313 - accuracy: 0.9301 - val_loss: 262.1656 -

The loss used is categorial cross entropy and the metric is accuracy

In [ ]:
#save the model in order to re-use the weights
name_model = 'vgg16_culture_customized.h5'
model.save(os.path.join('/content/gdrive/MyDrive',name_model))

In [ ]:
score = model.evaluate(test_data_unbalanced, verbose=False)
print('Test accuracy:', score[1])

Test accuracy: 0.6200000047683716
